In [1]:
import pandas as pd
import os
import glob

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "data/*.csv"))

In [29]:
dataframes = []

for f in csv_files:

    # read the csv file
    df = pd.read_csv(f)
    
    # skip setup files
    if len(df) <2:
        continue
    
    # add UUID as column
    df["uuid"] = os.path.splitext(f)[0][-36:]
    
    # append to array
    dataframes.append(df)

In [30]:
# concatenate data frames
merged = pd.concat(dataframes)

# save to svg
merged.to_csv("merged.csv")